In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD
import csv
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.decomposition import LatentDirichletAllocation

df = pd.read_csv('../dados_categorizados_completo_novo.csv')
df.head()

,Id,Enunciado,Tópico,Contexto
0,994,Meu primeiro programa\nEscreva um programa que...,Ambientação,Outros
1,996,Impressão de caracteres na tela\nEscreva um pr...,Ambientação,Outros
2,999,Impressão de caracteres na tela (Bart Simpson)...,Ambientação,Outros
3,1000,Operadores aritméticos\nQual o valor de X para...,Ambientação,Matemático
4,578,Operadores aritméticos \nQual o valor de Y par...,Ambientação,Matemático


In [2]:
df.Tópico.value_counts()

Estrutura condicional encadeada        161
Vetor                                  160
Variáveis e Estrutura Sequencial       157
Estrutura condicional composta         136
Matrizes                               134
Estrutura de repetição por contagem    117
Estrutura de repetição por condição    114
Strings                                 47
Ambientação                             19
Name: Tópico, dtype: int64

In [3]:
import spacy
nlp = spacy.load('pt')

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
X = df['Enunciado']
y = df['Tópico']

In [5]:
class LemmaTokenizer(object):
    def __call__(self, text):
        doc=nlp(text)
        tokens=[]
        for token in doc:
            if(token.text.isalpha() and len(token.text)>=2):
                tokens+=[token]
        return [t.lemma_ for t in tokens if t not in nlp.Defaults.stop_words]
pipeline= LinearSVC(
    penalty='l2',
    loss='squared_hinge',
    dual=True,
    tol=0.001,
    C=10.0,
    multi_class='ovr',
    fit_intercept=True,
    intercept_scaling=10,
    class_weight=None,
    verbose=0,
    random_state=None,
    max_iter=10000,
)

In [6]:
# from sklearn.model_selection import StratifiedKFold
# vetor_X_train=[]
# vetor_y_train=[]
# vetor_X_test=[]
# vetor_y_test=[]
# split = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
# for train_index, val_index in split.split(X, y):
#     vetor_X_train+=[X.iloc[train_index]]
#     vetor_X_test+=[X.iloc[val_index]]
#     vetor_y_train+=[y[train_index]]
#     vetor_y_test+=[y[val_index]]

In [7]:
# contador=0
# while(contador<len(vetor_X_train)):
#     X_train=vetor_X_train[contador]
#     y_train=vetor_y_train[contador]
#     X_test=vetor_X_test[contador]
    
#     text_clf_lsvc2 = Pipeline([('tfidf', TfidfVectorizer(stop_words=nlp.Defaults.stop_words,
#                             tokenizer=LemmaTokenizer(),sublinear_tf=True)),
#                             ('svd', TruncatedSVD(algorithm='randomized', n_components=300)),
#                             ('clf',pipeline),
#     ])
#     text_clf_lsvc2.fit(X_train, y_train)


#     contador+=1

In [8]:
text_clf_lsvc2 = Pipeline([('tfidf', TfidfVectorizer(stop_words=nlp.Defaults.stop_words,
                            tokenizer=LemmaTokenizer(),sublinear_tf=True)),
                            ('svd', TruncatedSVD(algorithm='randomized', n_components=300)),
                            ('clf',pipeline),
    ])
text_clf_lsvc2.fit(X, y)

/home/samuel/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['acercar', 'algum', 'antar', 'apoiar', 'baixar', 'caminhar', 'ceder', 'cincar', 'comer', 'conhecer', 'custar', 'dever', 'devir', 'direito', 'entrar', 'estevar', 'estivar', 'falto', 'formar', 'grupar', 'irar', 'ligar', 'mear', 'nado', 'nenhum', 'nó', 'o', 'obrigar', 'oitavar', 'orar', 'outro', 'parecer', 'parir', 'pelar', 'pontar', 'pôr', 'quantum', 'quartar', 'querer', 'quietar', 'quintar', 'segundar', 'seriar', 'sobrar', 'suar', 'tardar', 'terceirar', 'umar', 'vezar', 'vir', 'vário'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/home/samuel/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words={'acerca', 'ademais', 'adeus',
                                             'agora', 'ainda', 'algo',
                                             'alguma...
                                 tokenizer=<__main__.LemmaTokenizer object at 0x7f89810fad50>,
                                 use_idf=True, vocabulary=None)),
                ('svd',
                 TruncatedSVD(algorithm='randomized', n_components=300,
                  

In [9]:
def get_new_sentences(da,maximum,synonyms_lexicon):
    strings=[]
    for elemento in da['Enunciado']:
        if(len(da)+len(strings)<maximum):
            contador=0
            lista=elemento.split()
            while(contador<len(lista)):
                if(lista[contador] in synonyms_lexicon):
                    lista[contador]=synonyms_lexicon[lista[contador]][0]
                contador+=1
            strings+=[' '.join(lista)]
        else:
            break

    new_sentences = pd.Series(strings)
    return new_sentences


def get_synonyms_lexicon(path):
    synonyms_lexicon = {}
    text_entries = [l.strip() for l in open(path).readlines()]
    for e in text_entries:
        e = e.split(' ')
        k = e[0]
        v = e[1:len(e)]
        synonyms_lexicon[k] = v
    return synonyms_lexicon

synonyms_lexicon = get_synonyms_lexicon('../paráfrase/arq3.txt')

In [10]:
maximum = df.Tópico.value_counts().max()

# Separação de cada problemas em suas respectivas classes
g = df.groupby('Tópico')

ece = g.get_group('Estrutura condicional encadeada')
#     ec=g.get_group('Estrutura condicional')
vetor = g.get_group('Vetor')
ves = g.get_group('Variáveis e Estrutura Sequencial')
ecc = g.get_group('Estrutura condicional composta')
matrizes = g.get_group('Matrizes')
ercont = g.get_group('Estrutura de repetição por contagem')
ercond = g.get_group('Estrutura de repetição por condição')
strings = g.get_group('Strings')
ambientacao = g.get_group('Ambientação')

#criação de novas sentenças

new_ece = get_new_sentences(ece,maximum,synonyms_lexicon)
new_vetor = get_new_sentences(vetor,maximum,synonyms_lexicon)
new_ves = get_new_sentences(ves,maximum,synonyms_lexicon)
new_ecc = get_new_sentences(ecc,maximum,synonyms_lexicon)
new_matrizes = get_new_sentences(matrizes,maximum,synonyms_lexicon)
new_ercond = get_new_sentences(ercond,maximum,synonyms_lexicon)
new_ercont = get_new_sentences(ercont,maximum,synonyms_lexicon)
new_strings = get_new_sentences(strings,maximum,synonyms_lexicon)
new_ambientacao = get_new_sentences(ambientacao,maximum,synonyms_lexicon)


#criando dataframe de cada sentença criada

new_ece = pd.DataFrame(new_ece,columns=['Enunciado'])
new_ece['Tópico'] = 'Estrutura condicional'
new_ece['Contexto'] = 'Nada'


new_vetor = pd.DataFrame(new_vetor,columns=['Enunciado'])
new_vetor['Tópico'] = 'Vetor'
new_vetor['Contexto'] = 'Nada'


new_ves = pd.DataFrame(new_ves,columns=['Enunciado'])
new_ves['Tópico'] = 'Variáveis e Estrutura Sequencial'
new_ves['Contexto'] = 'Nada'


new_ecc = pd.DataFrame(new_ecc,columns=['Enunciado'])
new_ecc['Tópico'] = 'Estrutura condicional composta'
new_ecc['Contexto'] = 'Nada'


new_matrizes = pd.DataFrame(new_matrizes,columns=['Enunciado'])
new_matrizes['Tópico'] = 'Matrizes'
new_matrizes['Contexto'] = 'Nada'


new_ercont = pd.DataFrame(new_ercont,columns=['Enunciado'])
new_ercont['Tópico'] = 'Estrutura de repetição por contagem'
new_ercont['Contexto'] = 'Nada'


new_ercond = pd.DataFrame(new_ercond,columns=['Enunciado'])
new_ercond['Tópico'] = 'Estrutura de repetição por condição'
new_ercond['Contexto'] = 'Nada'


new_strings = pd.DataFrame(new_strings,columns=['Enunciado'])
new_strings['Tópico'] = 'Strings'
new_strings['Contexto'] = 'Nada'


new_ambientacao = pd.DataFrame(new_ambientacao,columns=['Enunciado'])
new_ambientacao['Tópico'] = 'Ambientação'
new_ambientacao['Contexto'] = 'Nada'


ece_extended = pd.concat([ece, new_ece], keys = ['Enunciado', 'Tópico'])
vetor_extended = pd.concat([vetor, new_vetor], keys = ['Enunciado', 'Tópico'])
ves_extended = pd.concat([ves, new_ves], keys = ['Enunciado', 'Tópico'])
ecc_extended = pd.concat([ecc, new_ecc], keys = ['Enunciado', 'Tópico'])
matrizes_extended = pd.concat([matrizes, new_matrizes], keys = ['Enunciado', 'Tópico'])
ercont_extended = pd.concat([ercont, new_ercont], keys = ['Enunciado', 'Tópico'])
ercond_extended = pd.concat([ercond, new_ercond], keys = ['Enunciado', 'Tópico'])
strings_extended = pd.concat([strings, new_strings], keys = ['Enunciado', 'Tópico'])
ambientacao_extended = pd.concat([ambientacao, new_ambientacao], keys = ['Enunciado', 'Tópico'])

#concantenando dataframes de sentenças parafráseadas a base de dados original 
df = pd.concat([ece_extended,vetor_extended,ves_extended,ecc_extended,
           matrizes_extended,ercont_extended,ercond_extended,strings_extended,ambientacao_extended])

/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:77: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:78: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:79: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'so

In [11]:
df.Tópico.value_counts()

Estrutura condicional composta         161
Estrutura de repetição por condição    161
Matrizes                               161
Vetor                                  161
Variáveis e Estrutura Sequencial       161
Estrutura condicional encadeada        161
Estrutura de repetição por contagem    161
Strings                                 94
Ambientação                             38
Name: Tópico, dtype: int64

In [12]:
cv=CountVectorizer(max_df=0.5,min_df=5,stop_words=nlp.Defaults.stop_words)
dtm=cv.fit_transform(df['Enunciado'])
LDA=LatentDirichletAllocation(n_components=500,random_state=42)
LDA.fit(dtm)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=500, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [13]:
teste=[]
for i,topic in enumerate(LDA.components_):
    teste+=[' '.join([cv.get_feature_names()[index] for index in topic.argsort()[-25:]])]
predictions=text_clf_lsvc2.predict(teste)

In [14]:
predictions

array(['Estrutura condicional composta',
       'Variáveis e Estrutura Sequencial',
       'Estrutura de repetição por contagem', 'Matrizes',
       'Estrutura condicional encadeada',
       'Variáveis e Estrutura Sequencial',
       'Estrutura de repetição por condição',
       'Estrutura de repetição por contagem', 'Vetor',
       'Estrutura condicional encadeada',
       'Estrutura condicional encadeada', 'Matrizes', 'Strings',
       'Variáveis e Estrutura Sequencial',
       'Estrutura de repetição por contagem', 'Vetor',
       'Estrutura de repetição por condição',
       'Variáveis e Estrutura Sequencial',
       'Variáveis e Estrutura Sequencial',
       'Variáveis e Estrutura Sequencial', 'Strings', 'Matrizes',
       'Variáveis e Estrutura Sequencial',
       'Estrutura de repetição por condição',
       'Variáveis e Estrutura Sequencial',
       'Estrutura de repetição por contagem',
       'Estrutura condicional encadeada',
       'Estrutura de repetição por contagem',
   

In [15]:
topic_results=LDA.transform(dtm)

In [16]:
y_pred=[]
for elemento in topic_results.argmax(axis=1):
    y_pred+=[predictions[elemento]]


In [17]:
df['LDA']=y_pred

In [18]:
df.LDA.value_counts()

Estrutura de repetição por condição    204
Vetor                                  182
Estrutura condicional encadeada        175
Variáveis e Estrutura Sequencial       159
Matrizes                               154
Estrutura de repetição por contagem    136
Estrutura condicional composta         130
Strings                                 94
Ambientação                             25
Name: LDA, dtype: int64

In [19]:
df.Tópico.value_counts()

Estrutura condicional composta         161
Estrutura de repetição por condição    161
Matrizes                               161
Vetor                                  161
Variáveis e Estrutura Sequencial       161
Estrutura condicional encadeada        161
Estrutura de repetição por contagem    161
Strings                                 94
Ambientação                             38
Name: Tópico, dtype: int64

In [20]:
df

Contexto                                          Enunciado  \
Enunciado 64  Matemático  De volta à área do triângulo\nEscreva um progr...   
          65  Matemático  Qual o tipo de triângulo?\nEscreva um programa...   
          66   Comercial  Qual o animal da cédula?\nAs cédulas do real c...   
          67     Consumo  Conta de energia\nEscreva um programa que dete...   
          68    Trânsito  Volume do Tanque de Combustível\nUm tanque de ...   
...                  ...                                                ...   
Tópico    14        Nada  Expressões aritméticas Qual assim preciosas ac...   
          15        Nada  Número sete Escreva acontecimentos sentido adi...   
          16        Nada  Impressão sentido poucas noções referente tal ...   
          17        Nada  Qual assim dobro? Escreva mil sistema durante ...   
          18        Nada  Impressão sentido poucas noções referente tal ...   

                  Id                           Tópico  \
Enunciado 64  1079.0  Estrutura condicional encadeada   
          65  1099.0  Estrutura condicional encadeada   
          66  1100.0  Estrutura condicional encadeada   
          67  1101.0  Estrutura condicional encadeada   
          68  1102.0  Estrutura condicional encadeada   
...              ...                              ...   
Tópico    14     NaN                      Ambientação   
          15     NaN                      Ambientação   
          16     NaN                      Ambientação   
          17     NaN                      Ambientação   
          18     NaN                      Ambientação   

                                              LDA  
Enunciado 64      Estrutura condicional encadeada  
          65      Estrutura condicional encadeada  
          66      Estrutura condicional encadeada  
          67      Estrutura condicional encadeada  
          68       Estrutura condicional composta  
...                                           ...  
Tópico    14                          Ambientação  
          15       Estrutura condicional composta  
          16                          Ambientação  
          17  Estrutura de repetição por condição  
          18                          Ambientação  

[1259 rows x 5 columns]

In [21]:
contexto=[]
for elemento in df['Contexto']:
    contexto+=[elemento]
lda=[]
for elemento in df['LDA']:
    lda+=[elemento]
topico=[]
for elemento in df['Tópico']:
    topico+=[elemento]

In [22]:
contador=0
y_test=[]
y_pred=[]
while(contador<len(contexto)):
    if(contexto[contador]!='Nada'):
        y_test+=[topico[contador]]
        y_pred+=[lda[contador]]
    contador+=1

In [23]:
print(metrics.confusion_matrix(y_test,y_pred))

[[ 15   1   1   0   1   0   1   0   0]
 [  0  90  16   5   0   3   0  18   4]
 [  0  14 137   0   2   1   0   7   0]
 [  0   1   0 101   4   4   0   2   2]
 [  0   0   3  12  94   1   1   4   2]
 [  0   0   2   2   2 123   1   0   4]
 [  0   0   2   2   0   1  41   1   0]
 [  0   3  10  21   2   2   0 115   4]
 [  0   0   2   5   4   1   0   1 147]]


In [24]:
print(metrics.classification_report(y_test,y_pred))

                                     precision    recall  f1-score   support

                        Ambientação       1.00      0.79      0.88        19
     Estrutura condicional composta       0.83      0.66      0.73       136
    Estrutura condicional encadeada       0.79      0.85      0.82       161
Estrutura de repetição por condição       0.68      0.89      0.77       114
Estrutura de repetição por contagem       0.86      0.80      0.83       117
                           Matrizes       0.90      0.92      0.91       134
                            Strings       0.93      0.87      0.90        47
   Variáveis e Estrutura Sequencial       0.78      0.73      0.75       157
                              Vetor       0.90      0.92      0.91       160

                           accuracy                           0.83      1045
                          macro avg       0.85      0.83      0.84      1045
                       weighted avg       0.83      0.83      0.83      10

In [25]:
print(metrics.accuracy_score(y_test,y_pred))

0.8258373205741627
